# Assignment #3
## P556: Applied Machine Learning

More often than not, we will use a deep learning library (Tensorflow, Pytorch, or the wrapper known as Keras) to implement our models. However, the abstraction afforded by those libraries can make it hard to troubleshoot issues if we don't understand what is going on under the hood. In this assignment you will implement a fully-connected and a convolutional neural network from scratch. To simplify the implementation, we are asking you to implement static architectures, but you are free to support variable number of layers/neurons/activations/optimizers/etc. We recommend that you make use of private methods so you can easily troubleshoot small parts of your model as you develop them, instead of trying to figure out which parts are not working correctly after implementing everything. Also, keep in mind that there is code from your fully-connected neural network that can be re-used on the CNN. 

Problem #1.1 (40 points): Implement a fully-connected neural network from scratch. The neural network will have the following architecture:

- Input layer
- Dense hidden layer with 512 neurons, using relu as the activation function
- Dropout with a value of 0.2
- Dense hidden layer with 512 neurons, using relu as the activation function
- Dropout with a value of 0.2
- Output layer, using softmax as the activation function

The model will use categorical crossentropy as its loss function. 
We will optimize the gradient descent using RMSProp, with a learning rate of 0.001 and a rho value of 0.9.
We will evaluate the model using accuracy.

Why this architecture? We are trying to reproduce from scratch the following [example from the Keras documentation](https://keras.io/examples/mnist_mlp/). This means that you can compare your results by running the Keras code provided above to see if you are on the right track.

In [13]:
import numpy as np
import keras
from keras.datasets import fashion_mnist
from sklearn.model_selection import KFold
from sklearn import preprocessing
class NeuralNetwork(object):
    def __init__(self,epochs, learning_rate,batch_size,input_dimension,output_dimension,dropout):
        self.epochs=epochs
        self.learning_rate=learning_rate
        self.neurons=512
        self.weight_1=np.random.randn(input_dimension,self.neurons)
        self.bias_1=np.zeros((1,self.neurons))
        self.weight_2=np.random.randn(self.neurons,self.neurons)
        self.bias_2=np.zeros((1,self.neurons))
        self.weight_3=np.random.randn(self.neurons,output_dimension)
        self.bias_3=np.zeros((1,output_dimension))
        self.batch_size=batch_size
        self.dropout=dropout

    def ReLu(self,x):
        return np.maximum(0,x)

    def Reluderivative(self,x):
        x[x<=0]=0
        x[x>0]=1
        return x


    def softmax(self,x):
        x = x - x.max(axis=1, keepdims=True)
        x_exp = np.exp(x)
        return x_exp / x_exp.sum(axis=1, keepdims=True)

    def cross_entropy(self,A,Y):
        cost = -np.mean(Y * np.log(A+ 1e-8))
        return cost

    def error(self,pred, actual):
        n = actual.shape[0]
        res = pred - actual
        return res/n

    def rmsprop(self,x,dx,rho=0.9):
        epslon=1e-8
        v=np.zeros(x.shape, dtype=float)
        v=rho * v + (1- rho) * dx **2
        x=x-self.learning_rate * (dx/ (np.sqrt(v)+epslon))
        return x


    def fit(self,X,Y):
        j=0
        while(j<self.epochs):
            i=0
            iteration=X.shape[0]
            while(i<iteration):
                    x=X[i:i+self.batch_size,:]
                    y=Y[i:i+self.batch_size,:]

                    result_1=np.matmul(x,self.weight_1)+self.bias_1
                    activation_1=self.ReLu(result_1)
                    dropout_arr_1=np.random.binomial(1,1-self.dropout,size=activation_1.shape)/1-self.dropout
                    activation_1=activation_1*dropout_arr_1

                    result_2=np.matmul(activation_1,self.weight_2)+self.bias_2
                    activation_2=self.ReLu(result_2)
                    dropout_arr_2=np.random.binomial(1,1-self.dropout,size=activation_2.shape)/1-self.dropout
                    activation_2=activation_2*dropout_arr_2

                    result_3=np.matmul(activation_2,self.weight_3)+self.bias_3
                    activation_3=self.softmax(result_3)

                    cross_entropy_loss=self.cross_entropy(activation_3,y)
                    #uncomment the following print statement to view cross entropy loss
                    #print("The cross entropy loss is",cross_entropy_loss)

                    activation_3_del=self.error(activation_3,y)
                    weight_3_del=np.matmul(activation_2.T,activation_3_del)
                    bias_3_del=activation_3_del

                    result_2_del=np.matmul(activation_3_del,self.weight_3.T)
                    activation_2_del=result_2_del * self.Reluderivative(activation_2)

                    weight_2_del=np.matmul(activation_1.T,activation_2_del)
                    bias_2_del=activation_2_del

                    result_1_del=np.matmul(activation_2_del,self.weight_2.T)
                    activation_1_del=result_1_del*self.Reluderivative(activation_1)

                    weight_1_del=np.matmul(x.T,activation_1_del)
                    bias_1_del=activation_1_del

                    self.weight_3=self.rmsprop(self.weight_3,weight_3_del,0.9)
                    self.bias_3=self.rmsprop(self.bias_3,bias_3_del,0.9)

                    self.weight_2=self.rmsprop(self.weight_2,weight_2_del,0.9)
                    self.bias_2=self.rmsprop(self.bias_2,bias_2_del,0.9)

                    self.weight_1=self.rmsprop(self.weight_1,weight_1_del,0.9)
                    self.bias_1=self.rmsprop(self.bias_1,bias_1_del,0.9)

                    i=i+self.batch_size
            j=j+1
            pass

    def evaluate(self,X):
        j=0
        a3_list=[]
        while(j<self.epochs):
                i=0
                iteration=X.shape[0]
                a3_argmax,a3_list=[],[]
                while(i<iteration):
                        x=X[i:i+self.batch_size,:]
                        result_1=np.matmul(x,self.weight_1)+self.bias_1
                        activation_1=self.ReLu(result_1)
                        dropout_arr_1=np.random.binomial(1,1-self.dropout,size=activation_1.shape)/1-self.dropout
                        activation_1=activation_1*dropout_arr_1

                        result_2=np.matmul(activation_1,self.weight_2)+self.bias_2
                        activation_2=self.ReLu(result_2)
                        dropout_arr_2=np.random.binomial(1,1-self.dropout,size=activation_2.shape)/1-self.dropout
                        activation_2=activation_2*dropout_arr_2

                        result_3=np.matmul(activation_2,self.weight_3)+self.bias_3
                        activation_3=self.softmax(result_3)

                        a3_list.append(np.argmax(activation_3,axis=1))
                        i=i+self.batch_size
                j=j+1
        a3_argmax=[]
        for elem in a3_list:
            a3_argmax.extend(elem)
        activation_3_argmax=np.asarray(a3_argmax)
        return activation_3_argmax


Problem #1.2 (10 points): Train your fully-connected neural network on the Fashion-MNIST dataset using 5-fold cross validation. Report accuracy on the folds, as well as on the test set.

In [14]:
import numpy as np
import keras
from keras.datasets import fashion_mnist
from sklearn.model_selection import KFold
from sklearn import preprocessing
# To simplify the usage of our dataset, we will be importing it from the Keras 
# library. Keras can be installed using pip: python -m pip install keras

# Original source for the dataset:
# https://github.com/zalandoresearch/fashion-mnist

# Reference to the Fashion-MNIST's Keras function: 
# https://keras.io/datasets/#fashion-mnist-database-of-fashion-articles
import keras,numpy
from keras.datasets import fashion_mnist

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[1], 'train samples')
print(x_test.shape[1], 'test samples')


# convert class vectors to binary class matrices
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train=preprocessing.normalize(x_train)
y_train=preprocessing.normalize(y_train)
x_test=preprocessing.normalize(x_test)
y_test=preprocessing.normalize(y_test)



nn=NeuralNetwork(20,0.001,1000,x_train.shape[1],y_train.shape[1],0.2)
cross_validation=KFold(n_splits=5,shuffle=False,random_state=None)
cross_validation_accuracy=[]
for train_index,test_index in cross_validation.split(x_train):
    x_train_cv,x_test_cv=x_train[train_index],x_train[test_index]
    y_train_cv,y_test_cv=y_train[train_index],y_train[test_index]
    nn.fit(x_train_cv,y_train_cv)
    activation_3_cv_argmax=nn.evaluate(x_test_cv)
    y_test_cv_argmax=np.argmax(y_test_cv,axis=1)
    accuracy=(activation_3_cv_argmax == y_test_cv_argmax).mean()
    accuracy=accuracy*100
    cross_validation_accuracy.append(accuracy)

for i in range(0,len(cross_validation_accuracy)):
    print("The cross validation accuracy of fold :",i+1,"is :",cross_validation_accuracy[i],"\n")

activation_3_test_argmax=nn.evaluate(x_test)
y_test_argmax=np.argmax(y_test,axis=1)
accuracy=(activation_3_test_argmax == y_test_argmax).mean()
print("Accuracy on test set is ",accuracy * 100)

784 train samples
784 test samples
The cross validation accuracy of fold : 1 is : 81.31666666666668 

The cross validation accuracy of fold : 2 is : 86.19166666666666 

The cross validation accuracy of fold : 3 is : 88.3 

The cross validation accuracy of fold : 4 is : 89.40833333333333 

The cross validation accuracy of fold : 5 is : 89.1 

Accuracy on test set is  80.60000000000001


Problem #2.1 (40 points): Implement a Convolutional Neural Network from scratch. Similarly to problem 1.1, we will be implementing the same architecture as the one shown in [Keras' CNN documentation](https://keras.io/examples/mnist_cnn/). That is:

- Input layer
- Convolutional hidden layer with 32 neurons, a kernel size of (3,3), and relu activation function
- Convolutional hidden layer with 64 neurons, a kernel size of (3,3), and relu activation function
- Maxpooling with a pool size of (2,2)
- Dropout with a value of 0.25
- Flatten layer
- Dense hidden layer, with 128 neurons, and relu activation function
- Dropout with a value of 0.5
- Output layer, using softmax as the activation function

Our loss function is categorical crossentropy and the evaluation will be done using accuracy, as in Problem 1.1. However, we will not be using the gradient optimizer known as Adadelta.

In [0]:
class ConvolutionalNeuralNetwork(object):
  def __init__(epochs, learning_rate):
    pass
  
  def fit(self):
    pass
  
  def evaluate(self):
    pass

Problem #2.2 (10 points): Train your convolutional neural network on the Fashion-MNIST dataset using 5-fold cross validation. Report accuracy on the folds, as well as on the test set.

In [0]:
import keras
from keras.datasets import fashion_mnist

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)